In [1]:
import pandas as pd

In [2]:
import numpy as np

In [3]:
import requests
import gzip
import os
import time
import dask.dataframe as dd
from dask.diagnostics import ProgressBar
import tempfile

# Solicitar el directorio de descarga al usuario
download_directory = input("Directorio donde descargar el archivo, terminado en /: ")

# Asegurarse de que el directorio ingresado termine con una barra diagonal
if not download_directory.endswith('/'):
    download_directory += '/'

# Definir la URL y el path local del archivo descargado
url = 'https://static.openfoodfacts.org/data/en.openfoodfacts.org.products.csv.gz'
local_gz_path = os.path.join(download_directory, 'TFM_OPF.csv.gz')

# Descargar el archivo .gz
print("Descargando el archivo .gz...")
response = requests.get(url, stream=True)
with open(local_gz_path, 'wb') as f:
    for chunk in response.iter_content(chunk_size=1024):
        f.write(chunk)

# Verificar si el archivo .gz se ha descargado correctamente
if not os.path.exists(local_gz_path):
    raise FileNotFoundError(f"El archivo {local_gz_path} no se descargó correctamente.")
else:
    print(f"El archivo {local_gz_path} se descargó correctamente.")

# Paso 2: Descomprimir el archivo .gz en fragmentos
def decompress_in_chunks(gz_path, output_path, chunk_size=1024*1024):
    with gzip.open(gz_path, 'rb') as f_in:
        with open(output_path, 'wb') as f_out:
            while True:
                chunk = f_in.read(chunk_size)
                if not chunk:
                    break
                f_out.write(chunk)
                print(f"Descomprimido {f_out.tell()} bytes")

# Usar un archivo temporal para la descompresión
with tempfile.NamedTemporaryFile(delete=False, suffix='.csv') as temp_csv_file:
    temp_csv_path = temp_csv_file.name

print(f"Descomprimiendo el archivo .gz a un archivo temporal: {temp_csv_path}")
decompress_in_chunks(local_gz_path, temp_csv_path)

# Verificar si el archivo .csv se ha descomprimido correctamente
if not os.path.exists(temp_csv_path):
    raise FileNotFoundError(f"El archivo {temp_csv_path} no se descomprimió correctamente.")
else:
    print(f"El archivo {temp_csv_path} se descomprimió correctamente.")

# Paso 3: Inspeccionar manualmente las primeras líneas del archivo CSV
print("Inspeccionando las primeras líneas del archivo CSV descomprimido:")
with open(temp_csv_path, 'r', encoding='utf-8') as f:
    for i in range(10):
        line = f.readline().strip()
        print(line)
        if not line:
            break

# Especifica el separador correcto
separator = '\t'  # Especifica el separador que utiliza tu archivo CSV
encoding = 'utf-8'

# Especificar los tipos de datos para las columnas problemáticas
dtype = {
    'additives_en': 'object',
    'additives_tags': 'object',
    'allergens': 'object',
    'brand_owner': 'object',
    'cities_tags': 'object',
    'emb_codes': 'object',
    'emb_codes_tags': 'object',
    'first_packaging_code_geo': 'object',
    'generic_name': 'object',
    'manufacturing_places': 'object',
    'manufacturing_places_tags': 'object',
    'no_nutrition_data': 'object',
    'packaging': 'object',
    'packaging_en': 'object',
    'packaging_tags': 'object',
    'packaging_text': 'object',
    'purchase_places': 'object',
    'serving_size': 'object',
    'stores': 'object',
    'traces': 'object',
    'traces_en': 'object',
    'traces_tags': 'object',
    'abbreviated_product_name': 'object',
    'owner': 'object',
    'additives_n': 'object',
    'cities': 'object',
    'nutriscore_score': 'object',
    'serving_quantity': 'object'
}

# Medir el tiempo de ejecución y usar ProgressBar para monitorear el progreso
start_time = time.time()

# Leer una pequeña muestra del archivo para determinar el delimitador y otros parámetros
sample_size = 10000  # Número de filas de la muestra

try:
    print("Leyendo una muestra del archivo CSV...")
    sample_df = dd.read_csv(temp_csv_path, blocksize=25e6, sample=sample_size, assume_missing=True, sep=separator, on_bad_lines='skip', dtype=dtype)
    # Mostrar información preliminar sobre el dataframe
    print(f"Columnas: {sample_df.columns.tolist()}")
    print(f"Número de columnas: {sample_df.shape[1]}")
except Exception as e:
    print(f"Error al leer el archivo CSV: {e}")

try:
    print("Leyendo el archivo CSV completo...")
    with ProgressBar():
        df = dd.read_csv(temp_csv_path, blocksize=25e6, assume_missing=True, sep=separator, encoding=encoding, on_bad_lines='skip', dtype=dtype)
        # Paso 5: Análisis Preliminar del Dataset
        print(f"Columnas: {df.columns.tolist()}")
        print(f"Número de filas: {df.shape[0].compute()}")
        print(f"Número de columnas: {df.shape[1]}")

        # Describir el dataset para obtener un resumen estadístico
        print(df.describe().compute())

        # Mostrar las primeras filas del dataset
        print(df.head())

        # Mostrar el tipo de datos de cada columna
        print(df.dtypes)
except Exception as e:
    print(f"Error al procesar el archivo CSV completo: {e}")

end_time = time.time()
print(f"Tiempo total de ejecución: {end_time - start_time} segundos")


Directorio donde descargar el archivo, terminado en /:  C:/TFM_OPF/


Descargando el archivo .gz...
El archivo C:/TFM_OPF/TFM_OPF.csv.gz se descargó correctamente.
Descomprimiendo el archivo .gz a un archivo temporal: C:\Users\crist\AppData\Local\Temp\tmpgpovconm.csv
Descomprimido 1048576 bytes
Descomprimido 2097152 bytes
Descomprimido 3145728 bytes
Descomprimido 4194304 bytes
Descomprimido 5242880 bytes
Descomprimido 6291456 bytes
Descomprimido 7340032 bytes
Descomprimido 8388608 bytes
Descomprimido 9437184 bytes
Descomprimido 10485760 bytes
Descomprimido 11534336 bytes
Descomprimido 12582912 bytes
Descomprimido 13631488 bytes
Descomprimido 14680064 bytes
Descomprimido 15728640 bytes
Descomprimido 16777216 bytes
Descomprimido 17825792 bytes
Descomprimido 18874368 bytes
Descomprimido 19922944 bytes
Descomprimido 20971520 bytes
Descomprimido 22020096 bytes
Descomprimido 23068672 bytes
Descomprimido 24117248 bytes
Descomprimido 25165824 bytes
Descomprimido 26214400 bytes
Descomprimido 27262976 bytes
Descomprimido 28311552 bytes
Descomprimido 29360128 bytes

C:\Users\crist\anaconda3\Lib\site-packages\dask\dataframe\io\csv.py:195: DtypeWarning: Columns (23,24,25) have mixed types. Specify dtype option on import or set low_memory=False.
  df = reader(bio, **kwargs)
C:\Users\crist\anaconda3\Lib\site-packages\dask\dataframe\io\csv.py:195: DtypeWarning: Columns (18,19,23,24,25,28,29,30,73,75,83,84) have mixed types. Specify dtype option on import or set low_memory=False.
  df = reader(bio, **kwargs)
C:\Users\crist\anaconda3\Lib\site-packages\dask\dataframe\io\csv.py:195: DtypeWarning: Columns (23,24,25) have mixed types. Specify dtype option on import or set low_memory=False.
  df = reader(bio, **kwargs)


[                                        ] | 0% Completed | 4.72 s

C:\Users\crist\anaconda3\Lib\site-packages\dask\dataframe\io\csv.py:195: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  df = reader(bio, **kwargs)
C:\Users\crist\anaconda3\Lib\site-packages\dask\dataframe\io\csv.py:195: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  df = reader(bio, **kwargs)
C:\Users\crist\anaconda3\Lib\site-packages\dask\dataframe\io\csv.py:195: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  df = reader(bio, **kwargs)


...

[######################################  ] | 97% Completed | 249.93 s

C:\Users\crist\anaconda3\Lib\site-packages\dask\dataframe\io\csv.py:195: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  df = reader(bio, **kwargs)
C:\Users\crist\anaconda3\Lib\site-packages\dask\dataframe\io\csv.py:195: DtypeWarning: Columns (23,24,25) have mixed types. Specify dtype option on import or set low_memory=False.
  df = reader(bio, **kwargs)


[########################################] | 100% Completed | 255.37 s
               code     created_t  last_modified_t  last_updated_t  \
count  3.236626e+06  3.236626e+06     3.236626e+06    3.213304e+06   
mean   4.622355e+49  1.603466e+09     1.655118e+09    1.708340e+09   
std    5.929508e+52  7.025122e+07     4.790258e+07    2.024911e+06   
min    1.000000e+00  1.328021e+09     1.353582e+09    1.705496e+09   
25%    2.251663e+12  1.587637e+09     1.649756e+09    1.707734e+09   
50%    5.051008e+12  1.635203e+09     1.680199e+09    1.707843e+09   
75%    8.992950e+12  1.698830e+09     1.714124e+09    1.717560e+09   
max    8.450851e+55  1.718258e+09     1.718258e+09    1.718258e+09   

       allergens_en  additives     nova_group  ecoscore_score  \
count           0.0        0.0  882066.000000   816530.000000   
mean            NaN        NaN       3.329151       49.050266   
std             NaN        NaN       1.047489       25.108720   
min             NaN        NaN       1

C:\Users\crist\anaconda3\Lib\site-packages\dask\dataframe\io\csv.py:195: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  df = reader(bio, **kwargs)


[########################################] | 100% Completed | 636.33 ms
          code                                                url  \
0        225.0  http://world-en.openfoodfacts.org/product/0000...   
1  207025004.0  http://world-en.openfoodfacts.org/product/0000...   
2    3429145.0  http://world-en.openfoodfacts.org/product/0000...   
3   26772226.0  http://world-en.openfoodfacts.org/product/0000...   
4         17.0  http://world-en.openfoodfacts.org/product/0000...   

          creator     created_t      created_datetime  last_modified_t  \
0  nutrinet-sante  1.623855e+09  2021-06-16T14:53:28Z     1.692102e+09   
1         kiliweb  1.656949e+09  2022-07-04T15:30:10Z     1.656949e+09   
2         kiliweb  1.630484e+09  2021-09-01T08:11:51Z     1.682646e+09   
3         kiliweb  1.654250e+09  2022-06-03T09:58:31Z     1.654270e+09   
4         kiliweb  1.529059e+09  2018-06-15T10:38:00Z     1.561464e+09   

  last_modified_datetime last_modified_by  last_updated_t  \
0   202

In [4]:
# Columnas a eliminar
columns_to_drop = [
    'url', 'creator', 'created_t', 'last_modified_t', 'last_modified_datetime', 'last_modified_by', 
    'last_updated_t', 'last_updated_datetime', 'abbreviated_product_name', 'packaging', 'packaging_tags', 
    'packaging_text', 'brands', 'categories', 'categories_tags', 'origins', 'origins_tags', 
    'manufacturing_places', 'labels', 'labels_tags', 'emb_codes', 'cities', 'countries', 'countries_tags', 
    'ingredients_text', 'allergens', 'traces', 'traces_tags', 'additives_n', 'additives', 'additives_tags', 
    'food_groups', 'food_groups_tags', 'states', 'states_tags', 'last_image_t', 'last_image_datetime', 
    'main_category', 'image_url', 'image_small_url', 'image_ingredients_url', 'image_ingredients_small_url', 
    'image_nutrition_url', 'image_nutrition_small_url'
]

# Medir el tiempo de ejecución y usar ProgressBar para monitorear el progreso
start_time = time.time()

try:
    print("Leyendo el archivo CSV completo...")
    with ProgressBar():
        df = dd.read_csv(temp_csv_path, blocksize=25e6, assume_missing=True, sep=separator, encoding=encoding, on_bad_lines='skip', dtype=dtype)
        
        # Eliminar las columnas especificadas
        df_cleaned = df.drop(columns=columns_to_drop, errors='ignore')
        
        # Verificar el número de columnas resultantes
        print(f"Número de columnas antes de eliminar: {len(df.columns)}")
        print(f"Número de columnas después de eliminar: {len(df_cleaned.columns)}")

        # Mostrar las columnas resultantes
        print(f"Columnas resultantes: {df_cleaned.columns.tolist()}")

        # Almacenar el DataFrame limpio en una variable
        cleaned_data = df_cleaned

except Exception as e:
    print(f"Error al procesar el archivo CSV completo: {e}")

end_time = time.time()
print(f"Tiempo total de ejecución: {end_time - start_time} segundos")

Leyendo el archivo CSV completo...
Número de columnas antes de eliminar: 206
Número de columnas después de eliminar: 162
Columnas resultantes: ['code', 'created_datetime', 'product_name', 'generic_name', 'quantity', 'packaging_en', 'brands_tags', 'categories_en', 'origins_en', 'manufacturing_places_tags', 'labels_en', 'emb_codes_tags', 'first_packaging_code_geo', 'cities_tags', 'purchase_places', 'stores', 'countries_en', 'ingredients_tags', 'ingredients_analysis_tags', 'allergens_en', 'traces_en', 'serving_size', 'serving_quantity', 'no_nutrition_data', 'additives_en', 'nutriscore_score', 'nutriscore_grade', 'nova_group', 'pnns_groups_1', 'pnns_groups_2', 'food_groups_en', 'states_en', 'brand_owner', 'ecoscore_score', 'ecoscore_grade', 'nutrient_levels_tags', 'product_quantity', 'owner', 'data_quality_errors_tags', 'unique_scans_n', 'popularity_tags', 'completeness', 'main_category_en', 'energy-kj_100g', 'energy-kcal_100g', 'energy_100g', 'energy-from-fat_100g', 'fat_100g', 'saturated

In [5]:
# Especificar los tipos de datos manualmente
dtype = {
    'additives_en': 'object',
    'additives_tags': 'object',
    'allergens': 'object',
    'brand_owner': 'object',
    'cities_tags': 'object',
    'emb_codes': 'object',
    'emb_codes_tags': 'object',
    'first_packaging_code_geo': 'object',
    'generic_name': 'object',
    'manufacturing_places': 'object',
    'manufacturing_places_tags': 'object',
    'no_nutrition_data': 'object',
    'packaging': 'object',
    'packaging_en': 'object',
    'packaging_tags': 'object',
    'packaging_text': 'object',
    'purchase_places': 'object',
    'serving_size': 'object',
    'stores': 'object',
    'traces': 'object',
    'traces_en': 'object',
    'traces_tags': 'object',
    'abbreviated_product_name': 'object',
    'owner': 'object',
    'additives_n': 'object',
    'cities': 'object',
    'nutriscore_score': 'object',
    'serving_quantity': 'object'
}

# Iniciar el temporizador
start_time = time.time()

try:
    print("Leyendo el archivo CSV completo...")
    with ProgressBar():
        # Leer el archivo CSV usando Dask con tipos de datos especificados
        df = dd.read_csv(temp_csv_path, blocksize=25e6, assume_missing=True, sep=separator, encoding=encoding, on_bad_lines='skip', dtype=dtype)
        
        # Eliminar las columnas especificadas
        df_cleaned = df.drop(columns=columns_to_drop, errors='ignore')
        
        # Verificar el número de columnas resultantes
        print(f"Número de columnas antes de eliminar: {len(df.columns)}")
        print(f"Número de columnas después de eliminar: {len(df_cleaned.columns)}")

        # Mostrar las columnas resultantes
        print(f"Columnas resultantes: {df_cleaned.columns.tolist()}")

        # Convertir el DataFrame de Dask a Pandas
        df_cleaned_pd = df_cleaned.compute()

        # Guardar el DataFrame resultante en un archivo CSV utilizando Pandas con codificación UTF-8
        output_csv_path = os.path.join(os.path.dirname(local_gz_path), 'dataset_columnas_eliminadas_pandas.csv')
        df_cleaned_pd.to_csv(output_csv_path, sep='\t', index=False, encoding='utf-8')
        print(f"Archivo CSV con columnas eliminadas guardado en: {output_csv_path}")

except Exception as e:
    print(f"Error al procesar el archivo CSV completo: {e}")

# Calcular y mostrar el tiempo total de ejecución
end_time = time.time()
print(f"Tiempo total de ejecución: {end_time - start_time} segundos")

# Verificar el número de filas y columnas del nuevo dataset
try:
    df_result = pd.read_csv(output_csv_path, sep='\t', low_memory=False, encoding='utf-8')
    num_filas = df_result.shape[0]
    num_columnas = df_result.shape[1]

    print(f"Número de filas del nuevo dataset: {num_filas}")
    print(f"Número de columnas del nuevo dataset: {num_columnas}")
except Exception as e:
    print(f"Error al leer el archivo CSV generado: {e}")

Leyendo el archivo CSV completo...
Número de columnas antes de eliminar: 206
Número de columnas después de eliminar: 162
Columnas resultantes: ['code', 'created_datetime', 'product_name', 'generic_name', 'quantity', 'packaging_en', 'brands_tags', 'categories_en', 'origins_en', 'manufacturing_places_tags', 'labels_en', 'emb_codes_tags', 'first_packaging_code_geo', 'cities_tags', 'purchase_places', 'stores', 'countries_en', 'ingredients_tags', 'ingredients_analysis_tags', 'allergens_en', 'traces_en', 'serving_size', 'serving_quantity', 'no_nutrition_data', 'additives_en', 'nutriscore_score', 'nutriscore_grade', 'nova_group', 'pnns_groups_1', 'pnns_groups_2', 'food_groups_en', 'states_en', 'brand_owner', 'ecoscore_score', 'ecoscore_grade', 'nutrient_levels_tags', 'product_quantity', 'owner', 'data_quality_errors_tags', 'unique_scans_n', 'popularity_tags', 'completeness', 'main_category_en', 'energy-kj_100g', 'energy-kcal_100g', 'energy_100g', 'energy-from-fat_100g', 'fat_100g', 'saturated

C:\Users\crist\anaconda3\Lib\site-packages\dask\dataframe\io\csv.py:195: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  df = reader(bio, **kwargs)
C:\Users\crist\anaconda3\Lib\site-packages\dask\dataframe\io\csv.py:195: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  df = reader(bio, **kwargs)
C:\Users\crist\anaconda3\Lib\site-packages\dask\dataframe\io\csv.py:195: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  df = reader(bio, **kwargs)


[                                        ] | 0% Completed | 4.06 s

C:\Users\crist\anaconda3\Lib\site-packages\dask\dataframe\io\csv.py:195: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  df = reader(bio, **kwargs)


[                                        ] | 0% Completed | 4.55 s

...

[######################################  ] | 97% Completed | 233.24 s

C:\Users\crist\anaconda3\Lib\site-packages\dask\dataframe\io\csv.py:195: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  df = reader(bio, **kwargs)


[####################################### ] | 98% Completed | 234.76 s

C:\Users\crist\anaconda3\Lib\site-packages\dask\dataframe\io\csv.py:195: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  df = reader(bio, **kwargs)


[########################################] | 100% Completed | 236.71 s
Archivo CSV con columnas eliminadas guardado en: C:/TFM_OPF\dataset_columnas_eliminadas_pandas.csv
Tiempo total de ejecución: 712.0728938579559 segundos
Número de filas del nuevo dataset: 3236626
Número de columnas del nuevo dataset: 162
